- `This code fetches the realtime vehicle records from RTD-Denver api and dumps these records to the postgresql table.` 

In [ ]:
## To postgresql
import logging
import requests
import psycopg2
from google.protobuf.json_format import MessageToJson
from gtfs_realtime_pb2 import FeedMessage, VehiclePosition
from gtfs_realtime_pb2 import VehicleDescriptor
from gtfs_realtime_pb2 import Position
from datetime import datetime
import time

logger = logging.getLogger(__name__)

class FeedPoller:
    def __init__(self, rtd_username, rtd_password):
        self.rtd_username = rtd_username
        self.rtd_password = rtd_password
        self.conn = psycopg2.connect(
            host="localhost",
            database="mydb",
            user="postgres",
            password="<put your postgres password here>"
        )

    def get_bus_positions(self):
        try:
            logger.info("Getting latest vehicle positions from RTD feed.")

            # to get latest vehicle positions you have to first sign up and get details
            username = "<put your rtd-denver.com username here(email)>"
            password = "<put your rtd-denver password here>"
            url = "https://www.rtd-denver.com/files/gtfs-rt/VehiclePosition.pb"

            response = requests.get(url, auth=(username, password))

            feed = FeedMessage()
            feed.ParseFromString(response.content)

            for entity in feed.entity:
                vehicle_position = entity.vehicle
                vehicle_id = vehicle_position.vehicle.id
                if len(vehicle_id) > 4:
                    location = {
                        "lat": vehicle_position.position.latitude,
                        "lon": vehicle_position.position.longitude
                    }

                    bus_position = {
                            "id": vehicle_position.vehicle.id,
                            "timestamp": vehicle_position.timestamp,
                            "location": location,
                            "bearing": vehicle_position.position.bearing
                    }

                    self.publish_to_postgresql(bus_position)

            logger.info("Published latest vehicle positions to PostgreSQL.")
        except Exception as e:
            logger.error(str(e))

    def publish_to_postgresql(self, speed):
        try:
            with self.conn.cursor() as cursor:
                # Assuming you have a table named "speed" with appropriate columns
                # Modify the query according to your table structure.
                query = "INSERT INTO speed (id, timestamp, latitude, longitude, bearing) VALUES (%s, %s, %s, %s, %s)"
                cursor.execute(query, (
                    speed["id"],
                    datetime.utcfromtimestamp(speed["timestamp"]),
                    speed["location"]["lat"],
                    speed["location"]["lon"],
                    speed["bearing"]
                ))

            self.conn.commit()
        except Exception as e:
            logger.error("Failed to insert into PostgreSQL: " + str(e))

if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    rtd_username = "<put your rtd-denver.com username here(email)>"
    rtd_password = "<put your rtd-denver password here>"

    feed_poller = FeedPoller(rtd_username, rtd_password)
    while True:
        feed_poller.get_bus_positions()
        time.sleep(30)
